# PyVista isosurface demos

In [1]:
import pyvista as pv
pv.set_jupyter_backend('trame')
from viz import build_validation_scenes, isosurface_with_clip,  generate_reg_mesh
from geometry import load_obj_mesh

In [2]:
levels = [ 10, 50, 100.0, 500.0, 1000.0]
scenes =  build_validation_scenes(reverse_faces=True)

In [3]:
#  single triangle
plotter = isosurface_with_clip(scenes[0], 
    levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x120761940_0&reconnect=auto" class="pyvista…

In [4]:
# two orthogonal triangles
plotter = isosurface_with_clip(scenes[1], levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x120baa210_1&reconnect=auto" class="pyvista…

In [5]:
# 4-triangle saddle
plotter = isosurface_with_clip(scenes[2], levels=levels, res=100)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x129734550_2&reconnect=auto" class="pyvista…

In [6]:
# 2 x 2 grid of 8 coplanar triangles, should look the same as 2 triangles, basic refinement test
mesh = generate_reg_mesh(lambda u, v: [u, v, 0], n=2)
plotter = isosurface_with_clip(mesh, levels=[10, 100, 200, 500, 1000], res=60)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x12072a5d0_3&reconnect=auto" class="pyvista…

In [7]:
# triangluated paraboloid segment, approximated by 8 x 8 grid, 128 triangles
mesh = generate_reg_mesh(lambda u, v: [u, v, (u-0.501)**2 + (v-0.501)**2], n=8)
plotter = isosurface_with_clip(mesh, levels=[10, 50, 100, 200, 500, 1000], res=60)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x12072aad0_4&reconnect=auto" class="pyvista…

In [8]:
# sharp corner, 4 triangles, 2 on each side
mesh = generate_reg_mesh(lambda u, v: [u, v, abs(10.01*u-5.01)], n=2)
plotter = isosurface_with_clip(mesh, levels=[10, 50, 100, 200, 500, 1000], res=150)
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x129737b10_5&reconnect=auto" class="pyvista…

In [ ]:
# ~ 280 triangle bunny, level sets chosen to look at the far behavior
mesh = load_obj_mesh("tests/Bunny-LowPoly.obj")
plotter = isosurface_with_clip(mesh, res=40, levels=[0.001, 0.01, 0.1])
plotter.show()

In [9]:
from viz import build_validation_scene_specs

scenes = build_validation_scene_specs(reverse_faces=False)
# pick a one-sided scene, e.g. tetrahedron
#scene = [s for s in scenes if s.name == "tetrahedron"][0]

In [14]:
plotter = isosurface_with_clip(scenes[4].mesh, 
    levels=[100, 200, 500, 1000], res=100, alpha=0.01, 
    one_sided=True,  
    include_faces=True,
    include_edges=False, 
    include_vertices= False,
    show_mesh=True
 )
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x13db4e0d0_8&reconnect=auto" class="pyvista…

In [21]:
plotter = isosurface_with_clip(scenes[5].mesh, 
    levels=[10, 20,  50, 100, 200, 500, 1000], res=60, alpha=0.1, 
    one_sided=True,  
    include_faces=True,
    include_edges=True, 
    include_vertices= True,
    show_mesh=True
 )
plotter.show()

Widget(value='<iframe src="http://localhost:55564/index.html?ui=P_0x1628b6d50_14&reconnect=auto" class="pyvist…

In [ ]:
[s.name for s in scenes]

In [ ]:
import numpy as np
import pyvista as pv

# -----------------------------
v = np.array([0.0, 0.0, 0.0])
q = np.array([0.0, 0.0, 1.0])
d = q - v

# Base polygon vertices (simple, concave) in plane z = -1, in cyclic order
A = np.array([
    [ 3.0,  0.0, -1.0],
    [ 0.0,  3.0, -1.0],
    [ 1.0,  1.0, -1.0],
    [-3.0,  0.0, -1.0],
    [ 0.0, -3.0, -1.0],
    [-1.0, -1.0, -1.0],
])

k = len(A)

# Points for the triangular surface mesh: index 0 is apex, indices 1..k are base vertices
points = np.vstack([v, A])

# Triangulated cone surface: triangles (v, a_i, a_{i+1})
faces = []
for i in range(k):
    i0 = 0
    i1 = 1 + i
    i2 = 1 + ((i + 1) % k)
    faces.extend([3, i0, i1, i2])  # "3" means triangle in PyVista face format
faces = np.array(faces, dtype=np.int64)

cone = pv.PolyData(points, faces)

# Color faces by the sign/value of d · (a_i × a_{i+1}) (same sign as d·n_i)
vals = np.zeros(k, dtype=float)
for i in range(k):
    ai = A[i]
    aj = A[(i + 1) % k]
    vals[i] = float(d @ np.cross(ai, aj))
cone.cell_data["d_dot_cross"] = vals

# -----------------------------
# Extra geometry for visualization
# -----------------------------
# Rays from apex to base vertices
rays = []
for i in range(k):
    rays.append(pv.Line(v, A[i]))

# Base polygon edges
base_edges = []
for i in range(k):
    base_edges.append(pv.Line(A[i], A[(i + 1) % k]))

# Markers for apex and q
apex_sphere = pv.Sphere(radius=0.08, center=v)
q_sphere = pv.Sphere(radius=0.08, center=q)

# Arrow showing d = q - v
arrow = pv.Arrow(start=v, direction=d, scale=1.0)

# -----------------------------
# Interactive plot
# -----------------------------
p = pv.Plotter()
p.add_axes()
p.add_text("Cone over concave polygon (faces colored by d · (a_i × a_{i+1}))", font_size=10)

p.add_mesh(
    cone,
    scalars="d_dot_cross",
    show_edges=True,
    opacity=0.7,
)

for ln in rays:
    p.add_mesh(ln, line_width=3)

for ln in base_edges:
    p.add_mesh(ln, line_width=3)

p.add_mesh(apex_sphere)
p.add_mesh(q_sphere)
p.add_mesh(arrow)

# A convenient view
p.view_isometric()
p.show()


In [ ]:
import numpy as np
import geometry
import viz

# _mesh_single_triangle
mesh_triangle = geometry.MeshData(
    V=np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]),
    faces=np.array([[0, 1, 2]], dtype=int),
)
#viz.visualize_pointed_vertices(mesh_triangle)

# build_validation_scene_specs() meshes (order in viz.py)
scenes = viz.build_validation_scene_specs()
#viz.visualize_pointed_vertices(scenes[0].mesh)  # triangle
#viz.visualize_pointed_vertices(scenes[1].mesh)  # two_faces
#viz.visualize_pointed_vertices(scenes[2].mesh)  # ring_up
#viz.visualize_pointed_vertices(scenes[3].mesh)  # ring_down

In [ ]:
viz.visualize_pointed_vertices(scenes[4].mesh)  # tetrahedron

In [ ]:
viz.visualize_pointed_vertices(scenes[5].mesh)  # cube_face_centers


In [ ]:
viz.visualize_pointed_vertices(scenes[6].mesh)  # double_cone_nonconvex_out

In [ ]:
viz.visualize_pointed_vertices(scenes[7].mesh)  # double_cone_nonconvex_both

In [ ]:
# tetrahedron from test_smoothed_offset_potential_numba_tetrahedron_one_sided_variants
V = np.array(
    [
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
    ]
)
faces = np.array(
    [
        [0, 1, 2],
        [1, 0, 3],
        [3, 0, 2],
        [2, 1, 3],
    ],
    dtype=int,
)
mesh_tet = geometry.MeshData(V=V, faces=faces)


In [ ]:
viz.visualize_pointed_vertices(mesh_tet)

In [ ]:
# flipped tetrahedron from test_smoothed_offset_potential_numba_flipped_normals
faces_flipped = faces[:, [0, 2, 1]]
mesh_tet_flipped = geometry.MeshData(V=V, faces=faces_flipped)
viz.visualize_pointed_vertices(mesh_tet_flipped)

In [ ]:
# flipped double_cone_nonconvex_* scenes
scenes_flipped = viz.build_validation_scene_specs(reverse_faces=True)
viz.visualize_pointed_vertices(scenes_flipped[6].mesh)

In [ ]:
viz.visualize_pointed_vertices(scenes_flipped[7].mesh)

In [ ]:
viz.visualize_pointed_vertices(scenes_flipped[5].mesh)